In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

#   勾配の不安定性を取り除く方法
#   1. 重み初期化法
keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")   #   正規分布でのHe初期化

he_avg_init = keras.initializers.VarianceScaling(scale=2., mode="fan_avg", distribution="uniform")  #   一様分布でのHe初期化かつfan_avgを利用
keras.layers.Dense(10, activation="sigmoid", kernel_initializer=he_avg_init)

#   2. 様々な活性化関数
model = keras.models.Sequential([
    [...]
    keras.layers.Dense(10, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(alpha=0.2),   #   LeakyReLU関数を適用したい層のすぐ後ろに追加
    [...]
])

layer = keras.layers.Dense(10, activation="selu", kernel_initialier="lecun_normal") #   SELU関数を適用する方法

#   3. バッチ正規化

#   活性化関数の後ろにBN層
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),                                      #   以下パラメータ数について
    keras.layers.BatchNormalization(),                                              #   28*28*4(BN層により入力当たり4つのパラメータ追加)
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),      #   28*28*300+300
    keras.layers.BatchNormalization(),                                              #   300*4
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),      #   300*100+100
    keras.layers.BatchNormalization(),                                              #   100*4
    keras.layers.Dense(10, activation="softmax")                                    #   100*10+10
])

#   活性化関数の前にBN層
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),                                      
    keras.layers.BatchNormalization(),                                              
    keras.layers.Dense(300, kernel_initializer="he_normal", use_bias=False),        #   BN層は入力ごとに1個のオフセットパラメータ(バイアス)を持っているのでuse_bias=Falseでバイアス項を取り除く      
    keras.layers.BatchNormalization(),                                         
    keras.layers.Activation("elu"),                                                 #   活性化関数を分離して記述     
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal", use_bias=False),      
    keras.layers.BatchNormalization(),                                              
    keras.layers.Dense(10, activation="softmax")                                    
])

#   4. 勾配クリッピング : バッチ正規化が使いにくい場合の代替手段となる
optimizer = keras.optimizers.SGD(clipvalue=1.0) #   勾配ベクトルを-1.0~1.0にクリッピング。clipnorm=1.0と指定すれば向きが変わらない。
model.compile(loss="mse", optimizer=optimizer)



In [ ]:
#   転移学習 : データセットが少ない時、既存のモデルを再利用して対処
model_A = keras.model.load_model("my_model_A.h5")   #   model_Aは8種類に分類するよう訓練したモデルと想定
model_B_on_A = keras.models.Sequential(model_A.layers[:-1]) #   model_Aの出力層以外をすべて再利用するモデルを定義
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))   #   ここでは二値分類を想定しているので、出力層は1つ

model_A_clone = keras.models.clone_model(model_A)   #   訓練によってmodel_Aの更新を避けたければこのようにクローンを作ってから再利用
model_A_clone.set_weights(model_A.get_weights())

for layer in model_B_on_A.layers[:-1]:  #   最初の数エポックは再利用層を凍結して新しい層に妥当な重みを学習するための時間を与える
    layer.trainable = False
model_B_on_A.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"]) #   凍結、凍結解除時はコンパイルが必要
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4, validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True  #   凍結解除
optimizer = keras.model.optimizer.SGD(lr=1e-4)  #   再利用層を壊さないために学習率を下げる
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16, validation_data=(X_valid_B, y_valid_B))


In [ ]:
#   optimizerの高速化

#   1. モーメンタム最適化 : 勾配降下法に加えて、加速度を導入する
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9)    #   重みの更新量を速度、momentumを加速度と考える。

#   2. NAG : モーメンタム最適化に加えて、運動量が働く方向に少し進んだ値(少し正確な値)を勾配測定に使用する
optimizer = keras.optimizer.SGD(lr=0.001, momentum=0.9, nesterov=True)

#   3. RMSProp : AdaGrad(最適値に近づくほど学習率(速度)を下げていく手法)のスローダウン問題を解決した手法
optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9)

#   4. Adam : モーメンタム最適化とRMSPropの組み合わせ
optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)


In [2]:
#   学習率のスケジューリング

#   1. パワースケジューリング : 減少加速度がどんどん小さくしていく
optimizer = keras.optimizers.SGD(lr=0.01, decay=1e-4)   #   lrは初期学習率、decayはステップ数の逆数

#   2. 指数スケジューリング : sステップごとに1/10になっていく
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1**(epoch/s)
    return exponential_decay_fn
exponential_decay_fn = exponential_decay(0.01, 20)

lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)  #   各エポックの冒頭で学習率を更新
history = model.fit(X_train_scaled, y_train, [...], callbacks=[lr_scheduler])   #   モデルの保存時、epochは保存されないので注意
                                                                                #   この問題はfit()のinitial_epochを適切に設定すれば解決する
#   3. 部分ごとに一定の学習率を決めておくスケジューリング
def piecewise_constant_fn(epoch):
    if epoch < 5:
        return 0.01
    elif epoch < 15:
        return 0.005
    else:
        return 0.001
lr_scheduler = keras.callbacks.LearningRateScheduler(piecewise_constant_fn)
history = model.fit([...])

#   4. 性能によるスケジューリング
#   5エポック連続で最良の検証損失が向上しなければ学習率に0.5をかける
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)

#   よりシンプルな設定方法
s = 20 * len(X_train)   #   20epoch実行したときのステップ数
learning_rate = keras.optimizers.schedules.ExponentialDecay(0.01, s, 0.1)
optimizer = keras.optimizers.SGD(learning_rate)


IndentationError: expected an indented block after function definition on line 8 (1815707354.py, line 9)

In [ ]:
#   正則化テクニック : ロス関数に重みの絶対値に比例して大きくなる正則化項を加えた関数の最小化を考えることで過学習を防ぐ

#   1. l1, l2正則化
layer = keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal", 
                           kernel_regularizer=keras.regularizers.l2(0.01))  #   正則化率0.01のl2正則化

#   modelを作成するときにデフォルト引数をあらかじめ設定しておく方法
from functools import partial
RegularizedDense = partial(keras.layers.Dense, activation="elu", kernel_initializer="he_normal", 
                           kernel_regularizer=keras.regularizer.l2(0.01))
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax", kernel_initializer="glorot_uniform")
])

#   2. ドロップアウト : 訓練中、rateの確率でニューロンを無視する。訓練後に重みに(1-p)倍するか、訓練中に(1-p)で割るかを行わなければいけない
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])

#   3. MCドロップアウト
y_probas = np.stack([model(X_test_scaled, training=True)    #   training=Trueでドロップアウト層をアクティブに
                     for sample in range(100)]) #   ドロップアウト訓練済みのモデルを使用して100回(MCサンプル数)テストした結果をスタックする
y_proba = y_probas.mean(axis=0)

#   4. 重み上限正則化
keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal", 
                   kernel_constraint=keras.constraints.max_norm(1.))    #   l2ノルム <= max_norm に制限